In [1]:
import lionagi as li

In [2]:
from pathlib import Path

data_path = Path.cwd() / "lionagi_data"  # Path to the data directory

### Prepare Data

In [3]:
# load files from directory
docs = li.load(input_dir=data_path, recursive=True, required_exts=[".py"])
docs = [doc for doc in docs if len(doc.content) > 100]

In [4]:
# # chunk
# pile = li.chunk(docs=docs, chunk_size=2000, overlap=0.1)

# embed
# await pile.embed_pile()

# # save
# pile.to_csv("lionagi_embedding.csv")

In [5]:
# reload pile from saved csv
pile = li.pile(csv_file="lionagi_embedding.csv")

### Construct Workflow

In [6]:
instruction = """
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name of a variable,
function, class, or module is used but not present in context,
you must check with the query engine. make sure to cross 
reference the code with the query engine to ensure the 
documentation is accurate.
"""

In [7]:
from PROMPTS import sys_prompt  # put your system prompt here

model = li.iModel(
    model="gpt-4o",
    provider="openai",
    interval_tokens=1_000_000,
    interval_requests=1_000,
    interval=60,
)

tools = pile.as_query_tool(
    name="qa_lionagi",
    guidance="Perform query to a QA bot",
    query_description="a term/phrase to lookup or a question to answer",
)

In [8]:
# await pile.query_pile("what is a pile in lionagi?")

In [9]:
branch = li.Branch(system=sys_prompt, tools=tools, imodel=model)

form = await branch.direct(
    instruction=instruction,
    context=docs[83].content,
    allow_action=True,
    allow_extension=True,
    verbose=True,
    max_extensions=2,
    retries=3,  # sometimes the model may fail to generate a valid response
)

Chatting with model...
Found action requests in model response. Processing actions...
Actions processed!

Found extension requests in model response.
------------------- Processing extension No.1 -------------------
Chatting with model...
Found action requests in model response. Processing actions...
Actions processed!
Analyzing action responses and generating answer...
------------------- Extension completed -------------------

Analyzing action responses and generating answer...

--------------------------------------------------------------
Directive successfully completed!


In [10]:
branch.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,188b898062e3e48fa107351f4fc94f70,System,2024-05-28T20:27:34.973901,system,"{'system_info': ' you are a helpful assistant,...",{'last_updated': {'recipient': '2024-05-28T20:...,system,bdab1133c58fb4cb50aeba9f0af56d8f
1,f27ff807491fdf242c6245d2794dc577,Instruction,2024-05-28T20:27:34.974756,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-28T20:27:...,user,bdab1133c58fb4cb50aeba9f0af56d8f
2,ec402e49ae4884b32281a5c4aa5ce96f,AssistantResponse,2024-05-28T20:27:41.859455,assistant,"{'assistant_response': '```json { ""answer"": ...",{'last_updated': {'sender': '2024-05-28T20:27:...,bdab1133c58fb4cb50aeba9f0af56d8f,user
3,4bf9ff8c9d9f5b75877fd575359e9d7d,ActionRequest,2024-05-28T20:27:41.861200,assistant,"{'action_request': {'function': 'qa_lionagi', ...",{'last_updated': {'function': '2024-05-28T20:2...,bdab1133c58fb4cb50aeba9f0af56d8f,2babb374402d72cc624a62e2ec384788
4,f6b18b7786e45ea8565b4caf44aba836,ActionResponse,2024-05-28T20:27:54.228381,assistant,"{'action_response': {'function': 'qa_lionagi',...",{'last_updated': {'function': '2024-05-28T20:2...,2babb374402d72cc624a62e2ec384788,bdab1133c58fb4cb50aeba9f0af56d8f
5,69a15972f230ea33cc3be0b6d9d14c9e,Instruction,2024-05-28T20:27:54.229352,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-28T20:27:...,user,bdab1133c58fb4cb50aeba9f0af56d8f
6,e25f7adb1c1b8772aedeac24c3f74162,AssistantResponse,2024-05-28T20:27:57.981681,assistant,"{'assistant_response': '```json { ""answer"": ...",{'last_updated': {'sender': '2024-05-28T20:27:...,bdab1133c58fb4cb50aeba9f0af56d8f,user
7,6097579aed2504e4979ef672c5eb2237,ActionRequest,2024-05-28T20:27:57.985443,assistant,"{'action_request': {'function': 'qa_lionagi', ...",{'last_updated': {'function': '2024-05-28T20:2...,bdab1133c58fb4cb50aeba9f0af56d8f,2babb374402d72cc624a62e2ec384788
8,5adfed4c740ff63e8d72a8e86a281988,ActionResponse,2024-05-28T20:28:08.027783,assistant,"{'action_response': {'function': 'qa_lionagi',...",{'last_updated': {'function': '2024-05-28T20:2...,2babb374402d72cc624a62e2ec384788,bdab1133c58fb4cb50aeba9f0af56d8f
9,a95c0e8e4b2c87660a7661c8d5c7b274,Instruction,2024-05-28T20:28:08.029046,user,{'instruction': 'please provide final answer b...,{'last_updated': {'sender': '2024-05-28T20:28:...,user,bdab1133c58fb4cb50aeba9f0af56d8f


In [11]:
form.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: 
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name of a variable,
function, class, or module is used but not present in context,
you must check with the query engine. make sure to cross 
reference the code with the query engine to ensure the 
documentation is accurate.

- Additional context: """
Copyright 2024 HaiyangLi
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""
# lionagi/core/session/directive_mixin.py
from lionagi.libs.ln_convert import strip_lower
from lionagi.core.unit import Unit

class DirectiveMixin:
    """
    Mixin class for handling chat interactions within the directive framework.
    """
    async def chat(
        self,
        instruction,  # additional instruction
        context=None,  # context to perform the instruction on
        system=None,  # optionally swap system message
        sender=None,  # sender of the instruction, default "user"
        recipient=None,  # recipient of the instruction, default "branch.ln_id"
        branch=None,
        requested_fields=None,  # fields to request from the context, default None
        form=None,  # form to create instruction from, default None,
        tools=False,  # the tools to use, use True to consider all tools, no tools by default
        invoke_tool=True,  # whether to invoke the tool when function calling, default True
        return_for...

**reason**: 
 Let's think step by step. To create accurate and comprehensive API documentation, it is essential to understand the meanings and descriptions of the terms used in the code. By querying the meanings of these terms, we can ensure that the documentation is precise and informative.

**actions**: 
 
 
1. **qa_lionagi**(query: Provide detailed meanings and descriptions for the following terms: 'Unit', 'directive', 'imodel', 'rulebook', 'branch', 'context', 'form', 'tools', 'annotation', 'retries', 'delay', 'backoff_factor', 'timeout', 'timing', 'return_branch', 'kwargs'.)

**action_required**: True

**extension_required**: True

**action_response**: 
 
 
1. **qa_lionagi**(query: Provide detailed meanings and descriptions for the following terms: 'Unit', 'directive', 'imodel', 'rulebook', 'branch', 'context', 'form', 'tools', 'annotation', 'retries', 'delay', 'backoff_factor', 'timeout', 'timing', 'return_branch', 'kwargs'.) 
 
 Here are detailed meanings and descriptions for the specified terms:

1. **Unit**: A class that combines functionalities from `Directive` and `DirectiveMixin`. It initializes with parameters like `branch`, `imodel`, `template`, and `rulebook`. It includes methods for handling chat interactions and directives.

2. **directive**: A command or instruction that the `Unit` class processes. It can be a string representing a specific action or set of actions to be executed.

3. **imodel**: An instance of the `iModel` class, which is used within the `Unit` class to handle model-related operations. It can be passed during initialization or accessed through the `branch`.

4. **rulebook**: A set of rules or guidelines used by the `Validator` class within the `Unit` to validate actions or inputs. It can be provided during initialization.

5. **branch**: An object that represents a specific branch or path in the workflow. It is used to manage and pass around the state and context within the `Unit`.

6. **context**: The surrounding information or environment in which a directive or chat interaction takes place. It can include previous messages, user inputs, and other relevant data.

7. **form**: A template or structure used to format and present data. In the `Unit` class, it refers to the `UnitForm` which is the default template for forms.

8. **tools**: Additional functionalities or utilities that can be invoked during the execution of a directive or chat interaction. They can be enabled or disabled based on the parameters.

9. **annotation**: Metadata or additional information that can be used to enhance or clarify the main data. In the context of the `Unit` class, it refers to whether annotations should be used during processing.

10. **retries**: The number of attempts to retry an operation in case of failure. It is part of the retry mechanism to handle transient errors.

11. **delay**: The amount of time to wait between retry attempts. It helps in spacing out retries to avoid overwhelming the system.

12. **backoff_factor**: A multiplier used to increase the delay between retries exponentially. It helps in gradually increasing the wait time after each failed attempt.

13. **timeout**: The maximum amount of time to wait for an operation to complete before considering it a failure. It ensures that operations do not hang indefinitely.

14. **timing**: The measurement or tracking of the duration of operations. It can be used for performance monitoring and optimization.

15. **return_branch**: A parameter that determines whether the branch object should be returned after processing a directive or chat interaction. It allows for further manipulation or inspection of the branch.

16. **kwargs**: A dictionary of additional keyword arguments that can be passed to functions or methods. It allows for flexible and extensible parameter passing., 
 
2. **qa_lionagi**(query: Provide detailed meanings and descriptions for the following terms: 'Unit', 'directive', 'imodel', 'rulebook', 'branch', 'context', 'form', 'tools', 'annotation', 'retries', 'delay', 'backoff_factor', 'timeout', 'timing', 'return_branch', 'kwargs'.) 
 
 Here are the detailed meanings and descriptions for the specified terms:

1. **Unit**: A class that serves as a directive and directive mixin, responsible for handling various operations such as chat interactions. It uses a form template and a validator for rule enforcement.

2. **directive**: A command or instruction given to the Unit class to perform specific actions. It can be a string representing the directive to be executed.

3. **imodel**: An instance of the iModel class, which is used within the Unit class to manage and interact with the model data.

4. **rulebook**: A set of rules or guidelines used by the Validator class to validate inputs and ensure they meet certain criteria.

5. **branch**: An object or instance that represents a specific branch or path within the Unit class. It can hold an imodel and other relevant data.

6. **context**: The surrounding information or environment in which a directive or instruction is executed. It provides additional details that may influence the execution of the directive.

7. **form**: A template or structure used to organize and present data within the Unit class. It can be customized or default to the UnitForm.

8. **tools**: Additional functionalities or utilities that can be invoked during the execution of a directive. They provide extended capabilities to the Unit class.

9. **annotation**: Metadata or additional information that can be used to enhance the understanding or processing of data within the Unit class.

10. **retries**: The number of attempts to re-execute a directive or operation in case of failure. It is part of the retry mechanism to ensure successful execution.

11. **delay**: The amount of time to wait between retry attempts. It helps in managing the timing of retries to avoid immediate successive failures.

12. **backoff_factor**: A multiplier used to increase the delay between retry attempts progressively. It helps in managing the retry timing more effectively by gradually increasing the wait time.

13. **timeout**: The maximum amount of time allowed for an operation or directive to complete. If the operation exceeds this time, it is aborted.

14. **timing**: The scheduling or timing details related to the execution of directives or operations. It includes aspects like delay, timeout, and retries.

15. **return_branch**: A flag or parameter indicating whether to return the branch object after executing a directive. It helps in managing the flow of data and control within the Unit class.

16. **kwargs**: A dictionary of additional keyword arguments that can be passed to functions or methods. It allows for flexible and dynamic parameter passing.

**available_tools**: ['qa_lionagi']

**answer**: 
 Here is the API documentation for the provided code:

---

### Class: `DirectiveMixin`

**Description**:
`DirectiveMixin` is a mixin class designed to handle chat interactions within the directive framework. It provides methods to process instructions with various parameters, interacting with tools, models, and validation rules as needed.

#### Method: `chat`

**Signature**:
```python
async def chat(
    self,
    instruction: str,
    context: Any = None,
    system: str = None,
    sender: str = "user",
    recipient: str = "branch.ln_id",
    branch: Branch = None,
    requested_fields: dict[str, str] = None,
    form: Any = None,
    tools: bool = False,
    invoke_tool: bool = True,
    return_form: bool = True,
    strict: bool = False,
    rulebook: Any = None,
    imodel: iModel = None,
    clear_messages: bool = False,
    use_annotation: bool = True,
    retries: int = 3,
    delay: float = 0,
    backoff_factor: float = 1,
    default: Any = None,
    timeout: float = None,
    timing: bool = False,
    return_branch: bool = False,
    **kwargs
) -> Any:
```

**Parameters**:
- `instruction` (str): Additional instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `system` (str, optional): Optionally swap the system message.
- `sender` (str, optional): Sender of the instruction, default is "user".
- `recipient` (str, optional): Recipient of the instruction, default is "branch.ln_id".
- `branch` (Branch, optional): The branch to use for the instruction.
- `requested_fields` (dict[str, str], optional): Fields to request from the context.
- `form` (Any, optional): Form to create instruction from, default is None.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `invoke_tool` (bool, optional): Whether to invoke the tool when function calling, default is True.
- `return_form` (bool, optional): Whether to return the form if a form is passed in, otherwise return a dict/str.
- `strict` (bool, optional): Whether to strictly enforce rule validation, default is False.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `clear_messages` (bool, optional): Whether to clear previous messages, default is False.
- `use_annotation` (bool, optional): Whether to use annotation as rule qualifier, default is True (needs rulebook if False).
- `retries` (int, optional): Number of retries if failed, default is 3.
- `delay` (float, optional): Number of seconds to delay before retrying, default is 0.
- `backoff_factor` (float, optional): Exponential backoff factor, default is 1 (no backoff).
- `default` (Any, optional): Default value to return if all retries failed.
- `timeout` (float, optional): Timeout for the call, default is None (no timeout).
- `timing` (bool, optional): If True, will return a tuple (output, duration), default is False.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `**kwargs`: Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the chat processing, format determined by the `return_form` parameter.

**Exceptions Raised**:
- `ValueError`: If an invalid combination of parameters is provided.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.chat(instruction="Hello", context={"data": "example"})
print(result)

# Example 2: Using additional parameters
result = await self.chat(
    instruction="Process data",
    context={"data": "example"},
    tools=True,
    retries=5,
    delay=2,
    backoff_factor=2
)
print(result)
```

#### Method: `direct`

**Signature**:
```python
async def direct(
    self,
    *,
    instruction: str = None,
    context: Any = None,
    form: Any = None,
    branch: Branch = None,
    tools: bool = None,
    return_branch: bool = False,
    reason: bool = False,
    predict: bool = False,
    score: Any = None,
    select: Any = None,
    plan: Any = None,
    allow_action: bool = False,
    allow_extension: bool = False,
    max_extension: int = None,
    confidence: Any = None,
    score_num_digits: int = None,
    score_range: Any = None,
    select_choices: Any = None,
    plan_num_step: int = None,
    predict_num_sentences: int = None,
    imodel: iModel = None,
    system: str = None,
    rulebook: Any = None,
    directive: str = None,
    **kwargs
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `form` (Any, optional): Form to create instruction from, default is None.
- `branch` (Branch, optional): The branch to use for the instruction.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `reason` (bool, optional): Whether to provide reasoning for the action, default is False.
- `predict` (bool, optional): Whether to predict the outcome, default is False.
- `score` (Any, optional): Scoring mechanism for the action.
- `select` (Any, optional): Selection mechanism for the action.
- `plan` (Any, optional): Planning mechanism for the action.
- `allow_action` (bool, optional): Whether to allow actions, default is False.
- `allow_extension` (bool, optional): Whether to allow extensions, default is False.
- `max_extension` (int, optional): Maximum number of extensions allowed, default is None.
- `confidence` (Any, optional): Confidence level for the action.
- `score_num_digits` (int, optional): Number of digits for the score, default is None.
- `score_range` (Any, optional): Range for the score, default is None.
- `select_choices` (Any, optional): Choices for selection, default is None.
- `plan_num_step` (int, optional): Number of steps for the plan, default is None.
- `predict_num_sentences` (int, optional): Number of sentences for prediction, default is None.
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `system` (str, optional): Optionally swap the system message.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `directive` (str, optional): Directive to process.
- `**kwargs`: Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the directive processing.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.direct(instruction="Analyze data", context={"data": "example"})
print(result)

# Example 2: Using additional parameters
result = await self.direct(
    instruction="Generate report",
    context={"data": "example"},
    tools=True,
    reason=True,
    predict=True,
    score=0.95,
    select="best_option",
    plan="detailed_plan"
)
print(result)
```

---

This documentation provides a comprehensive overview of the `DirectiveMixin` class and its methods, including detailed descriptions of parameters, return values, exceptions, and usage examples.

### Improve

In [12]:
edit = """
provide documentation only: final documentation in md 
format of the module of interest, do not include other 
fields do not present in JSON format, only markdown format 
you asked a lot of good questions and got plenty answers, 
please integrate your conversation, be a lot more technical, you will 
be rewarded with 500 dollars for great work, and 
punished for subpar work, take a deep breath, you can do it
"""

final_output = await branch.chat(instruction=edit)

In [13]:
from IPython.display import Markdown

Markdown(final_output)

# DirectiveMixin Module Documentation

## Class: `DirectiveMixin`

**Description**:
`DirectiveMixin` is a mixin class designed to handle chat interactions within the directive framework. It provides methods to process instructions with various parameters, interacting with tools, models, and validation rules as needed.

### Method: `chat`

**Signature**:
```python
async def chat(
    self,
    instruction: str,
    context: Any = None,
    system: str = None,
    sender: str = "user",
    recipient: str = "branch.ln_id",
    branch: Branch = None,
    requested_fields: dict[str, str] = None,
    form: Any = None,
    tools: bool = False,
    invoke_tool: bool = True,
    return_form: bool = True,
    strict: bool = False,
    rulebook: Any = None,
    imodel: iModel = None,
    clear_messages: bool = False,
    use_annotation: bool = True,
    retries: int = 3,
    delay: float = 0,
    backoff_factor: float = 1,
    default: Any = None,
    timeout: float = None,
    timing: bool = False,
    return_branch: bool = False,
    **kwargs
) -> Any:
```

**Parameters**:
- `instruction` (str): Additional instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `system` (str, optional): Optionally swap the system message.
- `sender` (str, optional): Sender of the instruction, default is "user".
- `recipient` (str, optional): Recipient of the instruction, default is "branch.ln_id".
- `branch` (Branch, optional): The branch to use for the instruction.
- `requested_fields` (dict[str, str], optional): Fields to request from the context.
- `form` (Any, optional): Form to create instruction from, default is None.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `invoke_tool` (bool, optional): Whether to invoke the tool when function calling, default is True.
- `return_form` (bool, optional): Whether to return the form if a form is passed in, otherwise return a dict/str.
- `strict` (bool, optional): Whether to strictly enforce rule validation, default is False.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `clear_messages` (bool, optional): Whether to clear previous messages, default is False.
- `use_annotation` (bool, optional): Whether to use annotation as rule qualifier, default is True (needs rulebook if False).
- `retries` (int, optional): Number of retries if failed, default is 3.
- `delay` (float, optional): Number of seconds to delay before retrying, default is 0.
- `backoff_factor` (float, optional): Exponential backoff factor, default is 1 (no backoff).
- `default` (Any, optional): Default value to return if all retries failed.
- `timeout` (float, optional): Timeout for the call, default is None (no timeout).
- `timing` (bool, optional): If True, will return a tuple (output, duration), default is False.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `**kwargs`: Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the chat processing, format determined by the `return_form` parameter.

**Exceptions Raised**:
- `ValueError`: If an invalid combination of parameters is provided.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.chat(instruction="Hello", context={"data": "example"})
print(result)

# Example 2: Using additional parameters
result = await self.chat(
    instruction="Process data",
    context={"data": "example"},
    tools=True,
    retries=5,
    delay=2,
    backoff_factor=2
)
print(result)
```

### Method: `direct`

**Signature**:
```python
async def direct(
    self,
    *,
    instruction: str = None,
    context: Any = None,
    form: Any = None,
    branch: Branch = None,
    tools: bool = None,
    return_branch: bool = False,
    reason: bool = False,
    predict: bool = False,
    score: Any = None,
    select: Any = None,
    plan: Any = None,
    allow_action: bool = False,
    allow_extension: bool = False,
    max_extension: int = None,
    confidence: Any = None,
    score_num_digits: int = None,
    score_range: Any = None,
    select_choices: Any = None,
    plan_num_step: int = None,
    predict_num_sentences: int = None,
    imodel: iModel = None,
    system: str = None,
    rulebook: Any = None,
    directive: str = None,
    **kwargs
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `form` (Any, optional): Form to create instruction from, default is None.
- `branch` (Branch, optional): The branch to use for the instruction.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `reason` (bool, optional): Whether to provide reasoning for the action, default is False.
- `predict` (bool, optional): Whether to predict the outcome, default is False.
- `score` (Any, optional): Scoring mechanism for the action.
- `select` (Any, optional): Selection mechanism for the action.
- `plan` (Any, optional): Planning mechanism for the action.
- `allow_action` (bool, optional): Whether to allow actions, default is False.
- `allow_extension` (bool, optional): Whether to allow extensions, default is False.
- `max_extension` (int, optional): Maximum number of extensions allowed, default is None.
- `confidence` (Any, optional): Confidence level for the action.
- `score_num_digits` (int, optional): Number of digits for the score, default is None.
- `score_range` (Any, optional): Range for the score, default is None.
- `select_choices` (Any, optional): Choices for selection, default is None.
- `plan_num_step` (int, optional): Number of steps for the plan, default is None.
- `predict_num_sentences` (int, optional): Number of sentences for prediction, default is None.
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `system` (str, optional): Optionally swap the system message.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `directive` (str, optional): Directive to process.
- `**kwargs`: Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the directive processing.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.direct(instruction="Analyze data", context={"data": "example"})
print(result)

# Example 2: Using additional parameters
result = await self.direct(
    instruction="Generate report",
    context={"data": "example"},
    tools=True,
    reason=True,
    predict=True,
    score=0.95,
    select="best_option",
    plan="detailed_plan"
)
print(result)
```

---

This documentation provides a comprehensive overview of the `DirectiveMixin` class and its methods, including detailed descriptions of parameters, return values, exceptions, and usage examples.

### check extension forms

In [14]:
if hasattr(form, "extension_forms"):
    for i in form.extension_forms:
        i.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: N/A
- Additional context: N/A
- Reason and prepare actions with GIVEN TOOLS ONLY.


**reason**: 
 Let's think step by step. To create accurate and comprehensive API documentation, it is essential to understand the meanings and descriptions of the terms used in the code. By querying the meanings of these terms, we can ensure that the documentation is precise and informative.

**actions**: 
 
 
1. **qa_lionagi**(query: Provide detailed meanings and descriptions for the following terms: 'Unit', 'directive', 'imodel', 'rulebook', 'branch', 'context', 'form', 'tools', 'annotation', 'retries', 'delay', 'backoff_factor', 'timeout', 'timing', 'return_branch', 'kwargs'.)

**action_required**: True

**action_response**: 

**available_tools**: ['qa_lionagi']

**answer**: 
 Here is the API documentation for the provided code:

---

### Class: `DirectiveMixin`

**Description**:
`DirectiveMixin` is a mixin class designed to handle chat interactions within the directive framework. It provides methods to process instructions with various parameters, interacting with tools, models, and validation rules as needed.

#### Method: `chat`

**Signature**:
```python
async def chat(
    self,
    instruction: str,
    context: Any = None,
    system: str = None,
    sender: str = "user",
    recipient: str = "branch.ln_id",
    branch: Branch = None,
    requested_fields: dict[str, str] = None,
    form: Any = None,
    tools: bool = False,
    invoke_tool: bool = True,
    return_form: bool = True,
    strict: bool = False,
    rulebook: Any = None,
    imodel: iModel = None,
    clear_messages: bool = False,
    use_annotation: bool = True,
    retries: int = 3,
    delay: float = 0,
    backoff_factor: float = 1,
    default: Any = None,
    timeout: float = None,
    timing: bool = False,
    return_branch: bool = False,
    **kwargs
) -> Any:
```

**Parameters**:
- `instruction` (str): Additional instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `system` (str, optional): Optionally swap the system message.
- `sender` (str, optional): Sender of the instruction, default is "user".
- `recipient` (str, optional): Recipient of the instruction, default is "branch.ln_id".
- `branch` (Branch, optional): The branch to use for the instruction.
- `requested_fields` (dict[str, str], optional): Fields to request from the context.
- `form` (Any, optional): Form to create instruction from, default is None.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `invoke_tool` (bool, optional): Whether to invoke the tool when function calling, default is True.
- `return_form` (bool, optional): Whether to return the form if a form is passed in, otherwise return a dict/str.
- `strict` (bool, optional): Whether to strictly enforce rule validation, default is False.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `clear_messages` (bool, optional): Whether to clear previous messages, default is False.
- `use_annotation` (bool, optional): Whether to use annotation as rule qualifier, default is True (needs rulebook if False).
- `retries` (int, optional): Number of retries if failed, default is 3.
- `delay` (float, optional): Number of seconds to delay before retrying, default is 0.
- `backoff_factor` (float, optional): Exponential backoff factor, default is 1 (no backoff).
- `default` (Any, optional): Default value to return if all retries failed.
- `timeout` (float, optional): Timeout for the call, default is None (no timeout).
- `timing` (bool, optional): If True, will return a tuple (output, duration), default is False.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `**kwargs`: Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the chat processing, format determined by the `return_form` parameter.

**Exceptions Raised**:
- `ValueError`: If an invalid combination of parameters is provided.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.chat(instruction="Hello", context={"data": "example"})
print(result)

# Example 2: Using additional parameters
result = await self.chat(
    instruction="Process data",
    context={"data": "example"},
    tools=True,
    retries=5,
    delay=2,
    backoff_factor=2
)
print(result)
```

#### Method: `direct`

**Signature**:
```python
async def direct(
    self,
    *,
    instruction: str = None,
    context: Any = None,
    form: Any = None,
    branch: Branch = None,
    tools: bool = None,
    return_branch: bool = False,
    reason: bool = False,
    predict: bool = False,
    score: Any = None,
    select: Any = None,
    plan: Any = None,
    allow_action: bool = False,
    allow_extension: bool = False,
    max_extension: int = None,
    confidence: Any = None,
    score_num_digits: int = None,
    score_range: Any = None,
    select_choices: Any = None,
    plan_num_step: int = None,
    predict_num_sentences: int = None,
    imodel: iModel = None,
    system: str = None,
    rulebook: Any = None,
    directive: str = None,
    **kwargs
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `form` (Any, optional): Form to create instruction from, default is None.
- `branch` (Branch, optional): The branch to use for the instruction.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `reason` (bool, optional): Whether to provide reasoning for the action, default is False.
- `predict` (bool, optional): Whether to predict the outcome, default is False.
- `score` (Any, optional): Scoring mechanism for the action.
- `select` (Any, optional): Selection mechanism for the action.
- `plan` (Any, optional): Planning mechanism for the action.
- `allow_action` (bool, optional): Whether to allow actions, default is False.
- `allow_extension` (bool, optional): Whether to allow extensions, default is False.
- `max_extension` (int, optional): Maximum number of extensions allowed, default is None.
- `confidence` (Any, optional): Confidence level for the action.
- `score_num_digits` (int, optional): Number of digits for the score, default is None.
- `score_range` (Any, optional): Range for the score, default is None.
- `select_choices` (Any, optional): Choices for selection, default is None.
- `plan_num_step` (int, optional): Number of steps for the plan, default is None.
- `predict_num_sentences` (int, optional): Number of sentences for prediction, default is None.
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `system` (str, optional): Optionally swap the system message.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `directive` (str, optional): Directive to process.
- `**kwargs`: Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the directive processing.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.direct(instruction="Analyze data", context={"data": "example"})
print(result)

# Example 2: Using additional parameters
result = await self.direct(
    instruction="Generate report",
    context={"data": "example"},
    tools=True,
    reason=True,
    predict=True,
    score=0.95,
    select="best_option",
    plan="detailed_plan"
)
print(result)
```

---

This documentation provides a comprehensive overview of the `DirectiveMixin` class and its methods, including detailed descriptions of parameters, return values, exceptions, and usage examples.

### check actions

In [15]:
print(f"The task invoked query engine {len(pile.query_response)} times.")

for i in pile.query_response:
    print(i.response)

The task invoked query engine 2 times.
Here are detailed meanings and descriptions for the specified terms:

1. **Unit**: A class that combines functionalities from `Directive` and `DirectiveMixin`. It initializes with parameters like `branch`, `imodel`, `template`, and `rulebook`. It includes methods for handling chat interactions and directives.

2. **directive**: A command or instruction that the `Unit` class processes. It can be a string representing a specific action or set of actions to be executed.

3. **imodel**: An instance of the `iModel` class, which is used within the `Unit` class to handle model-related operations. It can be passed during initialization or accessed through the `branch`.

4. **rulebook**: A set of rules or guidelines used by the `Validator` class within the `Unit` to validate actions or inputs. It can be provided during initialization.

5. **branch**: An object that represents a specific branch or path in the workflow. It is used to manage and pass around t